<a href="https://colab.research.google.com/github/pranavprajapati/house_prices_prediction/blob/master/datasets_check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##DATA EXPLORATION

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.1/spark-2.4.1-bin-hadoop2.7.tgz
!tar xf spark-2.4.1-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.1-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()


HI everyone

In [4]:
from google.colab import files
files.upload()


{}

In [1]:
!ls


sample_data


In [0]:
!rm -rf 

In [0]:
import pandas as pd
import numpy as np
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer
from pyspark.ml.regression import LinearRegression
from pyspark.ml.regression import RandomForestRegressor
from pyspark.sql.functions import corr
from pyspark.ml import Pipeline
from pyspark.sql.functions import isnan, when, count, col
from pyspark.sql.types import DoubleType
from pyspark.sql.types import IntegerType
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import GBTRegressor


In [0]:
df.filter((df["ValueRatio"] == "") | df["ValueRatio"].isNull() | isnan(df["ValueRatio"])).count()

35

In [0]:
df = spark.read.csv('cleaned_2016.csv',inferSchema = True, header =True)
#df.printSchema()

In [0]:

df = df.withColumn("regionidzip", df["regionidzip"].cast(IntegerType()))
df = df.withColumn("ValueRatio", df["regionidzip"].cast(DoubleType()))


In [0]:
df.printSchema()

root
 |-- parcelid: integer (nullable = true)
 |-- logerror: double (nullable = true)
 |-- airconditioningtypeid: double (nullable = true)
 |-- architecturalstyletypeid: double (nullable = true)
 |-- basementsqft: double (nullable = true)
 |-- bathroomcnt: double (nullable = true)
 |-- bedroomcnt: double (nullable = true)
 |-- buildingclasstypeid: double (nullable = true)
 |-- buildingqualitytypeid: double (nullable = true)
 |-- calculatedbathnbr: double (nullable = true)
 |-- finishedfloor1squarefeet: double (nullable = true)
 |-- calculatedfinishedsquarefeet: double (nullable = true)
 |-- finishedsquarefeet12: double (nullable = true)
 |-- finishedsquarefeet13: double (nullable = true)
 |-- finishedsquarefeet15: double (nullable = true)
 |-- finishedsquarefeet50: double (nullable = true)
 |-- finishedsquarefeet6: double (nullable = true)
 |-- fips: double (nullable = true)
 |-- fireplacecnt: double (nullable = true)
 |-- fullbathcnt: double (nullable = true)
 |-- garagecarcnt: double

In [0]:
vec_assembler = VectorAssembler(inputCols=['latitude',"landtaxvaluedollarcnt",'season','yearbuilt','bathroomcnt','regionidzip',"landtaxvaluedollarcnt",'censustractandblock','propertylandusetypeid','lotsizesquarefeet','house_age','bedroomcnt'], outputCol="features")


'taxamount','ValueRatio',"structuretaxvaluedollarcnt","landtaxvaluedollarcnt","TaxScore"

In [0]:
vec_assembler

VectorAssembler_b32fced7cd2c

In [0]:
linregout = vec_assembler.setHandleInvalid("skip").transform(df)

In [0]:
linreg_f = linregout.select("features","taxvaluedollarcnt")

In [0]:
linreg_f.show()

+--------------------+-----------------+
|            features|taxvaluedollarcnt|
+--------------------+-----------------+
|[3.428099E7,23741...|         360170.0|
|[3.4136312E7,5791...|         119906.0|
|[3.37558E7,73362....|         244880.0|
|[3.3485643E7,2649...|         434551.0|
|[3.3870089E7,1567...|        2447951.0|
|[3.3899475E7,4697...|         111521.0|
|[3.4207204E7,1990...|         306000.0|
|[3.35496E7,143230...|         210064.0|
|[3.36127E7,80983....|         190960.0|
|[3.4164671E7,5055...|         105954.0|
|[3.407222E7,53108...|        1090127.0|
|[3.4189804E7,4056...|          70119.0|
|[3.36709E7,202253...|         601000.0|
|[3.3913333E7,1762...|         254817.0|
|[3.3676787E7,2551...|         345023.0|
|[3.4176362E7,3120...|         480000.0|
|[3.37764E7,14083....|          70316.0|
|[3.3770148E7,1780...|         253138.0|
|[3.4164085E7,3072...|         416279.0|
|[3.3858594E7,1372...|         260249.0|
+--------------------+-----------------+
only showing top

In [0]:
training, test = linreg_f.randomSplit([0.8,0.2])

In [0]:

linreg = LinearRegression(featuresCol = 'features', labelCol = 'taxvaluedollarcnt')

In [0]:
linreg = linreg.fit(training)

In [0]:
linpred = linreg.evaluate(test)
linpred.predictions.show()

+--------------------+-----------------+------------------+
|            features|taxvaluedollarcnt|        prediction|
+--------------------+-----------------+------------------+
|[3.3340045E7,2447...|         545689.0|307768.25000008056|
|[3.3340937E7,5048...|         841483.0|  633742.890399165|
|[3.3400699E7,4332...|         151198.0|117141.75803738972|
|[3.3401249E7,2073...|         289911.0|302774.57267815573|
|[3.34074E7,85157....|         177418.0|133837.57197393104|
|[3.3408179E7,1916...|        2300000.0|  2531503.71356121|
|[3.3409415E7,4345...|         170870.0|116892.18593838718|
|[3.3410986E7,6885...|         933547.0|1005762.7720437376|
|[3.3412124E7,3213...|         426276.0|507640.06756739784|
|[3.3415205E7,4944...|         102039.0|130175.43674833188|
|[3.3416144E7,8571...|        1525391.0|1307335.1066968888|
|[3.3416775E7,6700...|         798970.0|  954189.612486517|
|[3.3417686E7,6075...|         886288.0| 883344.9265224049|
|[3.3417795E7,9731...|         110941.0|

In [0]:
eval = RegressionEvaluator(labelCol="taxvaluedollarcnt", predictionCol="prediction", metricName="rmse")
# Root Mean Square Error
rmse = eval.evaluate(linpred.predictions)
print("RMSE: %.3f" % rmse)
# Mean Square Error
mse = eval.evaluate(linpred.predictions, {eval.metricName: "mse"})
print("MSE: %.3f" % mse)
# Mean Absolute Error
mae = eval.evaluate(linpred.predictions, {eval.metricName: "mae"})
print("MAE: %.3f" % mae)
# r2 - coefficient of determination
r2 = eval.evaluate(linpred.predictions, {eval.metricName: "r2"})
print("r2: %.3f" %r2)

RMSE: 127931.821
MSE: 16366550885.618
MAE: 71732.796
r2: 0.945


In [0]:
# Initialize model with columns to utilize
rf = RandomForestRegressor(featuresCol="features",
                           labelCol="taxvaluedollarcnt",
                           predictionCol="Prediction",
                           seed=42
                           )

In [0]:
modelreg = rf.fit(training)

In [0]:
predictions_rf = model.transform(test)

In [0]:
predictions.select("Prediction", "taxvaluedollarcnt").show(5)

+------------------+-----------------+
|        Prediction|taxvaluedollarcnt|
+------------------+-----------------+
| 323648.6151672956|         545689.0|
| 688971.2296311021|         841483.0|
|167704.96113102607|         151198.0|
| 319978.9816182234|         289911.0|
| 208977.8719481871|         177418.0|
+------------------+-----------------+
only showing top 5 rows



In [0]:
# Select columns to compute test error
evaluator = RegressionEvaluator(labelCol="taxvaluedollarcnt", 
                                predictionCol="Prediction")

rmse = evaluator.evaluate(predictions, {evaluator.metricName: "rmse"})
r2 = evaluator.evaluate(predictions, {evaluator.metricName: "r2"})
# Print Model Metrics
print('RMSE: ' + str(rmse))
print('R^2: ' + str(r2))

RMSE: 253640.91110554672
R^2: 0.7844872969912521


In [0]:
# Train a Gradient Boosted Trees (GBT) model.
gbt = GBTRegressor(featuresCol="features",
                           labelCol="taxvaluedollarcnt",
                           predictionCol="Prediction",
                           seed=42
                           )

# Train model.
modelgbt = gbt.fit(training)


In [0]:
predictions_gbt = model.transform(test)
predictions_gbt.select("Prediction_Price", "taxvaluedollarcnt").show(5)


+------------------+-----------------+
|  Prediction_Price|taxvaluedollarcnt|
+------------------+-----------------+
|343790.76194412855|         545689.0|
|   621256.18789959|         841483.0|
|  79931.2140980965|         151198.0|
| 331401.0364094571|         289911.0|
| 144093.9132617582|         177418.0|
+------------------+-----------------+
only showing top 5 rows



In [0]:
# Select columns to compute test error
evaluator = RegressionEvaluator(labelCol="taxvaluedollarcnt", 
                                predictionCol="Prediction_Price")

rmse = evaluator.evaluate(predictions_gbt, {evaluator.metricName: "rmse"})
r2 = evaluator.evaluate(predictions_gbt, {evaluator.metricName: "r2"})
# Print Model Metrics
print('RMSE: ' + str(rmse))
print('R^2: ' + str(r2))

RMSE: 218101.3976445796
R^2: 0.8406503323003021


In [0]:
# Select columns to compute test error
evaluator = RegressionEvaluator(labelCol="taxvaluedollarcnt", 
                                predictionCol="Prediction_Price")
# Dictionary of model predictions to loop over
models = {'Gradient Boosted Trees': predictions_gbt, 'Random Forest Regression': predictions_rf}
for key, preds in models.items():
  # Create evaluation metrics
  rmse = evaluator.evaluate(preds, {evaluator.metricName: "rmse"})
  r2 = evaluator.evaluate(preds, {evaluator.metricName: "r2"})

  # Print Model Metrics
  print(key + ' RMSE: ' + str(rmse))
  print(key + ' R^2: ' + str(r2))

Gradient Boosted Trees RMSE: 218101.3976445796
Gradient Boosted Trees R^2: 0.8406503323003021
Random Forest Regression RMSE: 218101.3976445796
Random Forest Regression R^2: 0.8406503323003021


In [0]:
featuresCol = list(df.columns)
fi_df = pd.DataFrame(modelgbt.featureImportances.toArray(), columns = ["Importance"])
fi_df['Feature'] = pd.Series(featuresCol)
fi_df.sort_values(by=["Importance"], ascending=False, inplace=True)

# Inspect Results
fi_df.head(10)

,Importance,Feature
1,0.328538,logerror
4,0.175176,basementsqft
3,0.088986,architecturalstyletypeid
9,0.087383,calculatedbathnbr
0,0.069305,parcelid
7,0.052497,buildingclasstypeid
10,0.050870,finishedfloor1squarefeet
5,0.050736,bathroomcnt
11,0.036420,calculatedfinishedsquarefeet
8,0.033127,buildingqualitytypeid
